# 1. 로그인

In [1]:
import pickle
import time
import pandas as pd
from pykiwoom.kiwoom import *
import warnings
warnings.filterwarnings('ignore')

kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

In [2]:
# login - information
account_num = kiwoom.GetLoginInfo('ACCOUNT_CNT') # number of account
accounts = kiwoom.GetLoginInfo('ACCNO') # list of account
user_id = kiwoom.GetLoginInfo('USER_ID') # user id
user_name = kiwoom.GetLoginInfo('USER_NAME') # user name
#print('number of account : {}'.format(account_num))

In [3]:
state = kiwoom.GetConnectState()
if state == 0:
    print("미연결")
elif state == 1:
    print("연결완료")

연결완료


# 2. 변수 설정


In [1]:
from datetime import date,datetime,timedelta
today = date.today()+timedelta(hours=9)
today = today.isoformat()
today = today.replace('-','')
set_d = today
set_d

'20220728'

In [5]:
#SectorList : {sector_name: stockcode_list}
sector = {'Food':['097950', '271560', '000080','004370','005300'],
          'Clothing' : ['383220','093050','020000','105630','001070'],
          'Chemical' : ['051910','096770','010950','051900','090430'],
          'Medicine' : ['207940','068270','302440','000100','128940'],
          'Non_Metal' : ['003670','003410','010780','300720'],
          'Metal': ['005490','010130', '004020','016380','001230'],
          'Machine' : ['034020','018880','241560','112610'],
          'Electronic' : ['005930','000660','373220','006400','066570'],
          'Construction' : ['000720','006360','047040','051600'],
          'Transport': ['011200','003490','086280','180640','028670'],
          'Distribution' : ['028260','023530','282330','139480','004170'],
          'Power': ['015760','036460','017390'],
          'Tele' : ['017670','030200','032640'],
          'Finance': ['323410','003550','000810','377300','006800'],
          'Brokerage' : ['005940','016360','008560','039490'],
          'Insurer':['032830','005830','000060','001450'],
          'Service':['035420','035720','259960','018260','036570'],
          'Manufacturer':['005380','000270','012330','329180','033780','009150']     
     }

In [6]:
df_sector = {'Food':{},
          'Clothing' : {},
          'Chemical' : {},
          'Medicine' : {},
          'Non_Metal' : {},
          'Metal': {},
          'Machine' : {},
          'Electronic' : {},
          'Construction' : {},
          'Transport': {},
          'Distribution' : {},
          'Power': {},
          'Tele' : {},
          'Finance': {},
          'Brokerage' : {},
          'Insurer': {},
          'Service': {},
          'Manufacturer': {}}

# 3. 데이터 수집 함수
## 3-1. 일별 종가 데이터

In [7]:
def checkStockPrice(sector_name,stockcode):
    dfs = [] #데이터프레임리스트
    count=0
    df = kiwoom.block_request("opt10081",
                          종목코드=stockcode,
                          기준일자=set_d,
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)
    count+=1
    print('데이터 수집 시작.. ({}~)'.format(df.loc[0,'일자']))
    print('데이터 수집 중.. (~{})'.format(df.loc[len(df)-1,'일자']))
    dfs.append(df)
    while kiwoom.tr_remained:
        df = kiwoom.block_request("opt10081",
                                  종목코드=stockcode,
                                  기준일자=set_d,
                                  수정주가구분=1,
                                  output="주식일봉차트조회",
                                  next=2)
        dfs.append(df)
        time.sleep(1)
        print('데이터 수집 중.. (~{})'.format(df.loc[len(df)-1,'일자']))
        count+=1
        if kiwoom.tr_remained == False:
            print('데이터 수집 완료')
        elif count == 5:
            print('3000개 데이터를 확보')
            break;
        
    df_item = pd.concat(dfs)
    df_item.reset_index(drop=True,inplace=True)
    df_sector[sector_name][stockcode] = df_item

## 3-2 일별 코스피 지수 / 개별(사용안함)

In [8]:
'''
def checkKospi():
    dfk = []
    count = 0
    df = kiwoom.block_request("opt20006",
                          업종코드="001",
                          기준일자=set_d,
                          output="업종일봉조회",
                          next=0)
    dfk.append(df)
    print('코스피 데이터 수집 시작.. ({}~)'.format(df.loc[0,'일자']))
    print('코스피 수집 중.. (~{})'.format(df.loc[len(df)-1,'일자']))
    count+= 1
    
    while kiwoom.tr_remained:
        df = kiwoom.block_request("opt20006",
                                  업종코드="001",
                                  기준일자=set_d,
                                  output="업종일봉조회",
                                  next=2)
        time.sleep(1)
        print('코스피 데이터 수집 중.. (~{})'.format(df.loc[len(df)-1,'일자']))
        count+=1
        dfk.append(df)
        if kiwoom.tr_remained == False:
            print('데이터 수집 완료')
        elif count == 5:
            print('3000개 코스피 데이터를 확보')
            break;
        
    df = pd.concat(dfk)
    df.reset_index(drop = True, inplace=True)
    return df
    
''' 
    

# 4. 데이터 수집 자동화

In [9]:
#일봉 데이터 수집 자동화 코드
for sector_name, stockcode_list in sector.items():
    print(f'---------{sector_name}업종 데이터 수집----------')
    for stockcode in stockcode_list:
        # 주식항목이름
        stockname = kiwoom.GetMasterCodeName(stockcode)
        print(f'--------[{stockname}] 항목 데이터 수집---------')
        checkStockPrice(sector_name,stockcode)
        
    

---------Food업종 데이터 수집----------
--------[CJ제일제당] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170901)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150331)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121019)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100526)
3000개 데이터를 확보
--------[오리온] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170901)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170707)
데이터 수집 완료
--------[하이트진로] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170901)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150331)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121019)
CommRqData opt10081 opt10081 2 0101


TypeError: 'NoneType' object is not subscriptable

데이터 수집 중.. (~20100526)
3000개 데이터를 확보
--------[농심] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170901)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150331)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121019)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100526)
3000개 데이터를 확보
--------[롯데칠성] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170901)
CommRqData opt10081 opt10081 2 0101


TypeError: 'NoneType' object is not subscriptable

데이터 수집 중.. (~20150331)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121019)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100526)
3000개 데이터를 확보
---------Clothing업종 데이터 수집----------
--------[F&F] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20210521)
--------[LF] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20170901)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20150331)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20121019)
CommRqData opt10081 opt10081 2 0101
데이터 수집 중.. (~20100526)
3000개 데이터를 확보
--------[한섬] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
데이터 수집 시작.. (20220721~)
데이터 수집 중.. (~20200219)
CommRqData opt10081 opt10081 2 0101


...

데이터 수집 중.. (~20121019)
CommRqData opt10081 opt10081 2 0101


TypeError: 'NoneType' object is not subscriptable

TypeError: 'NoneType' object is not subscriptable

TypeError: 'NoneType' object is not subscriptable

데이터 수집 중.. (~20100526)
3000개 데이터를 확보


In [10]:
# 코스피 지수 데이터 수집
#kospi = checkKospi()

# 5. 데이터 개수 확인


In [11]:
len(df_sector)

18

In [12]:
df_sector['Food']['271560'].columns

Index(['종목코드', '현재가', '거래량', '거래대금', '일자', '시가', '고가', '저가', '수정주가구분', '수정비율',
       '대업종구분', '소업종구분', '종목정보', '수정주가이벤트', '전일종가'],
      dtype='object')

In [13]:
df_sector['Food']['271560'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   종목코드     1239 non-null   object
 1   현재가      1239 non-null   object
 2   거래량      1239 non-null   object
 3   거래대금     1239 non-null   object
 4   일자       1239 non-null   object
 5   시가       1239 non-null   object
 6   고가       1239 non-null   object
 7   저가       1239 non-null   object
 8   수정주가구분   1239 non-null   object
 9   수정비율     1239 non-null   object
 10  대업종구분    1239 non-null   object
 11  소업종구분    1239 non-null   object
 12  종목정보     1239 non-null   object
 13  수정주가이벤트  1239 non-null   object
 14  전일종가     1239 non-null   object
dtypes: object(15)
memory usage: 72.7+ KB


# 6. 특성선택

In [14]:
# df_sector = {'Food':{'code': DataFrame}}
for sector_name, df_dict in df_sector.items():
    for stockcode, df in df_dict.items():
        df = df[['일자','현재가','거래량','시가','고가','저가']]
        df['일자'] = pd.to_datetime(df['일자'])
        df['종가'] = df['현재가'].astype(int)
        del df['현재가']
        df['거래량'] = df['거래량'].astype(int)
        df['시가'] = df['시가'].astype(int)
        df['고가'] = df['고가'].astype(int)
        df['저가'] = df['저가'].astype(int)
        df_sector[sector_name][stockcode] = df

# 7. 데이터 저장

In [15]:
with open('df_sector.pickle','wb') as f:
    pickle.dump(df_sector,f)

In [16]:
with open('sector.pickle','wb') as f:
    pickle.dump(sector,f)